In [2]:
import bregnn.ffwd as ffwd 

Using TensorFlow backend.


In [3]:
import bregnn.io as io

In [4]:
import os

features = ['Jet_pt',
 'Jet_eta',
 'rho',
 'Jet_mt',
 'Jet_leadTrackPt',
 'Jet_leptonPtRel',
 'Jet_leptonDeltaR',
 'Jet_neHEF',
 'Jet_neEmEF',
 'Jet_vtxPt',
 'Jet_vtxMass',
 'Jet_vtx3dL',
 'Jet_vtxNtrk',
 'Jet_vtx3deL',
 'Jet_energyRing_dR0_em_Jet_e',
 'Jet_energyRing_dR1_em_Jet_e',
 'Jet_energyRing_dR2_em_Jet_e',
 'Jet_energyRing_dR3_em_Jet_e',
 'Jet_energyRing_dR4_em_Jet_e',
 'Jet_energyRing_dR0_neut_Jet_e',
 'Jet_energyRing_dR1_neut_Jet_e',
 'Jet_energyRing_dR2_neut_Jet_e',
 'Jet_energyRing_dR3_neut_Jet_e',
 'Jet_energyRing_dR4_neut_Jet_e',
 'Jet_energyRing_dR0_ch_Jet_e',
 'Jet_energyRing_dR1_ch_Jet_e',
 'Jet_energyRing_dR2_ch_Jet_e',
 'Jet_energyRing_dR3_ch_Jet_e',
 'Jet_energyRing_dR4_ch_Jet_e',
 'Jet_energyRing_dR0_mu_Jet_e',
 'Jet_energyRing_dR1_mu_Jet_e',
 'Jet_energyRing_dR2_mu_Jet_e',
 'Jet_energyRing_dR3_mu_Jet_e',
 'Jet_energyRing_dR4_mu_Jet_e',
 'Jet_numDaughters_pt03']

columns = features + ['Jet_mcPt']

base_dir = os.environ['SCRATCH'] + '/bregression'

data = io.read_data(base_dir+'/ttbar_unweighted_full80M_selected.hd5', columns = columns)

In [15]:
reload(ffwd)

X = data[features].values
y = (data['Jet_mcPt']/data['Jet_pt']).values.reshape(-1,1)

reg = ffwd.FFWDRegression('ffwd',X.shape[1:],const_output_biases=np.array([1.,0.8]))

model = reg(True)

<class 'bregnn.losses.RegularizedGaussNll'> True


In [16]:
# ?Input

In [17]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ffwd_inp (InputLayer)        (None, 35)                0         
_________________________________________________________________
ffwd_bn0 (BatchNormalization (None, 35)                140       
_________________________________________________________________
ffwd_dense1 (Dense)          (None, 256)               8960      
_________________________________________________________________
ffwd_bn1 (BatchNormalization (None, 256)               1024      
_________________________________________________________________
ffwd_do1 (Dropout)           (None, 256)               0         
_________________________________________________________________
ffwd_dense2 (Dense)          (None, 256)               65536     
_________________________________________________________________
ffwd_bn2 (BatchNormalization (None, 256)               1024      
__________

In [18]:
from keras.optimizers import Adam

In [19]:
# import bregnn.losses as losses

# opt = Adam(lr=1e-4)

# reload(losses)

# model.compile(optimizer=opt,loss=losses.gauss_nll)
# # model.compile(optimizer=opt,loss='mse')

In [20]:
from sklearn.model_selection import train_test_split

X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.05)

In [27]:
model.fit(X_train,y_train,validation_data=(X_valid,y_valid))

Train on 5299993 samples, validate on 278948 samples
Epoch 1/1
  55840/5299993 [..............................] - ETA: 20:05 - loss: 25.3197

KeyboardInterrupt: 

In [ ]:
# print(X_train.shape,y_train.shape,X_valid.shape,y_valid.shape)

In [ ]:
# y_pred = model.predict(X_valid[:10000])

In [ ]:
# ( (y_valid[:10000] - y_pred[:10000,0])**2/y_pred[:10000,1]**2 ) .max()

In [24]:
reg.fit(X_train,y_train,epochs=1)

<class 'bregnn.losses.RegularizedGaussNll'> True
Epoch 1/1
   2371/5299993 [..............................] - ETA: 9:09:31 - loss: 0.0082

KeyboardInterrupt: 

In [25]:
print(reg.model)

In [26]:
print(model)